# Notebook 1: Creating Items for Building Complexes

This notebook implements the first step of creating of the Klosterdatenbank-to-Factgrid-Workflow which is to create Items for the building complexes. It contains describing elements about the underlying data model and the workflow in general, as well as specific instructions in order to run the notebook. Markdown cells containing describing elements are marked as `#description`. Instructional sections are marked as `#instruction`.

Strictly speaking, the monastery database does not contain dedicated information on building complexes. Information on where a religious community had its place of operation is stored in the `gs_monastery_location` table. This table assigns each row of a religious community (`gsn_id`) to a location (`place_id`) and, if known, specific coordinates within this location (`longitude`, `latitude`). Such an assignment implies that the community lived or worked at this location at a certain point in time. At this point, we make the central assumption that a building complex of some kind, consisting of at least one building, must have existed. Accordingly, the building complexes created in this step represent both a row from the `gs_monastery_location` table and thus an assignment of a monastery to a specific location, as well as physical buildings in which religious communities worked and which may have continued to exist before or after their use and have experienced other use scenarios.

## Preparations

The notebook requires the following libraries to run. If an error occurs, make sure the libraries are installed on your system.

In [31]:
import pandas as pd
import numpy as np
import os
import csv

First, the export files are loaded into [Dataframes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). The dataframes are stored in a dictionary with the keys being the filenames, for easier access.

In [32]:
# Load Access exports
from helper_functions import load_files_from_folder

export_files = load_files_from_folder("data/exports_monasteryDB", "xlsx")

# Create dataframes for each table
dataframes = {key: pd.read_excel(value) for key, value in export_files.items()}

# Add dataframe for monasteries in factGrid (stored in a different directory)
dataframes["building_complexes_in_factgrid"] = pd.read_csv("data/factgrid_data/building_complexes_in_factgrid.csv")
dataframes["monasteries_in_factgrid"] = pd.read_csv("data/factgrid_data/monasteries_in_factgrid.csv")

Since `gs_monastery_location` does not contain the name of the monasteries, the table is joined to `gs_monastery` to extract the missing information. The resulting table is cut down to the relevant columns. The resulting dataframe is being filtered to only contain information on religious comunities that have the status "online", meaning they are not currently worked on anymore. Finally, to make sure that no duplicate building complexes are being created, the table is filtered against the existing building complexes in FactGrid.

In [33]:
# Merge gs_monastery_location and gs_monastery
merged_df = pd.merge(dataframes["gs_monastery_location"], dataframes["gs_monastery"], left_on='gsn_id', right_on='id_gsn', how='left')
# Filter for status 'online'
online_df = merged_df[merged_df["status"] == "Online"]
# Define columns to drop
drop_columns = [
    "relocated", 
    "comment", 
    "main_location", 
    "diocese_id", 
    "id_monastery", 
    "date_created", 
    "created_by_user", 
    "note", 
    "patrocinium",
    "selection", 
    "processing_status", 
    "gs_persons", 
    "selection_criteria", 
    "last_change", 
    "changed_by_user", 
    "founder",
    "Unnamed: 0_x",
    "Unnamed: 0_y"
]
# Prepare dataframe by dropping unnecessary columns
prepared_df = online_df.drop(drop_columns, axis="columns")
prepared_df = prepared_df[~prepared_df["id_monastery_location"].isin(dataframes["building_complexes_in_factgrid"]["GSVocabTerm"])]
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,51.681667,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande"
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,48.563200,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,51.295222,NaN,3790,Online,Zisterzienserinnenkloster Roßleben
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ..."
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ..."
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,49.805300,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,52.152778,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim"
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,53.053089,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande"
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,51.666767,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,52.916129,NaN,814,Online,"Kollegiatstift St. Andreas, Verden"


To double-check potential duplicates, the following cell finds buildings complexes that are connected to monasteries already existent in FactGrid. If the resulting DataFrame is empty, all building complexes will be linked to newly created monastery items.

In [34]:
existing_monasteries = prepared_df[prepared_df["gsn_id"].isin(dataframes["monasteries_in_factgrid"]["KlosterdatenbankID"])]
existing_monasteries

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ..."
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ..."
12,1397,1195,3017,975,1000.0,Ende 10. Jahrhundert,1802,NaN,NaN,7.892106,49.966703,Bingen,3017,Online,"Kollegiatstift St. Martin, Bingen"
14,10634,46480749,4878,1600,NaN,NaN,1791,NaN,NaN,7.490364,48.542733,Molsheim,4878,Online,"Kartause Molsheim, Frankreich"


## Labels

It is expected that items in FactGrid have a label in at least one language. While the FactGrid ID (also referred to as the "Q-Number") uniquely identifies the item, the label serves to capture the name of the item in everyday language. The label is also indexed for text-based search. The naming of the items created in this project follows the following rule:
- For the religious communities, the name from the monastery database is used as the label, for example "Zisterzienserkloster Georgenzell".
- For the building complexes, the labels are constructed according to the following schema: `Gebäudekomplex <monastery_name> [(<location_name>)]`. Here, `monastery_name` is again the name of the religious community from the `gs_monastery` table. `location_name` is a column of the `gs_monastery_location` table. In this column, if available, the specific name given to this location is stored. 

For example, the "Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien" (GSN [11665](https://klosterdatenbank.adw-goe.de/gsn/11665)) has two locations in the Belgian town of Sint-Truiden, namely the location "Sint Truiden" and the location "Metsteren" (see Figure). The constructed labels are then "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Sint-Truiden)" and "Gebäudekomplex Benediktinerinnenkloster Mielen, Sint-Truiden, Belgien (Metsteren)". However, location names are not available in all these cases, which can lead to duplicates in the labels. These are displayed in the workflow, so that location names can be added to distinguish them better.

<img src="documentation-images/Standorte GSN11665.png" alt="Monastery Locations of GSN 11665" width="500">

*Figure 1: Building Complexes of the Benedictine nun's monastery Mielen in Sint-Truiden, Belgium (GSN 11665). Base-Layer: OpenStreetMap.*

The following cell constructs the location names and saves them in a new column called "Lde" (see [Quickstatements specification](https://www.wikidata.org/wiki/Help:QuickStatements#Adding_labels,_aliases,_descriptions_and_sitelinks)).

In [35]:
# 1. Create new column with labels
prepared_df['Lde'] = "Gebäudekomplex " + prepared_df["monastery_name"].str.cat(prepared_df["location_name"].fillna(''), sep=" (") +")"
# 2. If necessary, delete empty brackets at end of labels
prepared_df['Lde'] = prepared_df["Lde"].str.replace(r'\(\)', '', regex=True).apply(lambda x: f'\"{x.strip()}\"')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,51.681667,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe..."
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,48.563200,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ..."
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,51.295222,NaN,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl..."
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien..."
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien..."
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,49.805300,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa..."
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,52.152778,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,..."
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,53.053089,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn..."
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,51.666767,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder..."
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,52.916129,NaN,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve..."


As mentioned above, there might be duplicate labels in cases where locations don't have an explicit name. Since they still can be distinguished from another by their identifier and coordinates, this is not necessarily a problem. However, the following cell will create a list of all the duplicate labels so that they can be examined.

**In order to resolve the duplicates**

1. Open and inspect the table located at `data/intermediate_results/duplicate_building_complex_labels.xslx`
2. Add location names in the monastery database
3. Create new exports from the monastery database and replace `data/exports_monasteryDB/gs_monastery.xlsx` and `data/exports_monasteryDB/gs_monastery_location.xlsx` with the new files
4. Re-run the notebook. The cell below now should no longer contain the duplicates you resolved. 

In [36]:
duplicated_building_complex_labels = prepared_df[prepared_df.duplicated(subset="Lde", keep=False)]
duplicated_building_complex_labels.to_excel('data/intermediate_results/duplicate_building_complex_labels.xlsx')
duplicated_building_complex_labels

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde
15,13700,46479207,8478,1496,NaN,NaN,1600,NaN,NaN,5.567778,50.646111,NaN,8478,Online,"Chorfrauen vom Heiligen Grab, Nieuwstadt, Nied...","""Gebäudekomplex Chorfrauen vom Heiligen Grab, ..."
16,13701,46483206,8478,1486,NaN,NaN,1496,NaN,NaN,5.860630,51.036193,NaN,8478,Online,"Chorfrauen vom Heiligen Grab, Nieuwstadt, Nied...","""Gebäudekomplex Chorfrauen vom Heiligen Grab, ..."


### Translation of Labels

FactGrid is a multilingual platform. Therefore, the labels for the monasteries and building complexes should not only be created in German, but also in English. Due to the heterogeneity of the monastery names in the database, a rule-based translation is difficult to implement. Instead, a Large-Language Model was used. The model, prompting, and details of the translation are described in more detail in the notebook "1a - Translation". We are using the [GWDG/KISSKI API](https://docs.hpc.gwdg.de/services/chat-ai/index.html), so in order to execute the notebook, a [SAIA API key](https://docs.hpc.gwdg.de/services/saia/index.html) is needed. Since the translation process can take some time, it has been outsourced to a separate notebook.

In [37]:
to_translate = prepared_df[["monastery_name", 'Lde']].copy()
to_translate["note"] = "x"
to_translate["Dde"] = "x"
to_translate = to_translate.rename(columns={"Lde": "building_Lde", "Dde": "building_Dde", "monastery_name" : "monastery_Lde", "note": "monastery_Dde"})
to_translate.to_csv("data/translation/to_translate.csv")
to_translate

,monastery_Lde,building_Lde,monastery_Dde,building_Dde
0,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...",x,x
1,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...",x,x
2,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...",x,x
3,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...",x,x
4,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...",x,x
5,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...",x,x
6,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...",x,x
7,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...",x,x
8,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...",x,x
9,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...",x,x


After executing the above cell, a table is generated in `data/translation` that contains all terms that should be translated: `to_translate.csv`. Execute Notebook 1a. Once the execution is completed, there should be a file named `translated.csv` that contains the translations within the `data/translation` folder. Once the file exists, you can run the next cell to load the translated labels.

In [38]:
translated = pd.read_csv("data/translation/translated.csv")
prepared_df["Len"] = translated["building_Len"].apply(lambda x:f'\"{x}\"')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde,Len
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,51.681667,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion..."
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,48.563200,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen"""
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,51.295222,NaN,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben"""
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian..."
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n..."
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,49.805300,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch..."
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,52.152778,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic..."
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,53.053089,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti..."
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,51.666767,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of..."
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,52.916129,NaN,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea..."


## Geocoordinates

Our data model separates religious communities from the building complexes in which they lived and worked. The geocoordinates of a location of a religious community are properties of the building complex in this modeling. In the monastery database, there are two levels of accuracy with which the localization of a monastery location can be performed: coordinates for a monastery location will either represent the exact point where the building was located, or the central point of a place, e.g. a village, in which it was located. It is to be noted that the centroid-based location always only represents an approximation of the centroid of the modern location. In cases where the exact location of the building complex is unknown, the respective item will not be linked to any coordinates. Instead, the coordinates of the place where it is located should be queried. In all other cases, the coordinates are directly linked to the building complexes, using values from the `latitude` and `longitude` columns as [P48](https://database.factgrid.de/wiki/Property:P48).

In [39]:
for index, row in prepared_df.iterrows():
    if (not pd.isna(row["latitude"])) and (not pd.isna(row["longitude"])):
        prepared_df.loc[index, "P48"] = f'@{row["latitude"]}/{row["longitude"]}'
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde,Len,P48
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,51.681667,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,48.563200,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,51.295222,NaN,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n...",NaN
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,49.805300,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,52.152778,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,53.053089,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,51.666767,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,52.916129,NaN,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792


## Connection to places

The prerequisite for connecting all building complexes with the locations in which they were found is that there are items in FactGrid for these locations. For the collection on locality data in the monastery database, the open source service [geonames](https://www.geonames.org/) was the central tool. Therefore, there is a geonames ID in the monastery database for each location. In FactGrid, there is also a qualifier (P418) for the GeoNames ID. This can be used to assign the location data to each other and to subsequently fill in missing locations. The notebook 1b - Place Matching describes this process.

In order to match all places needed, a matching between FactGrid and the place data from the monastery database is needed. All information that is already available should be placed in a file called `places_reconciled.xlsx` in the `reconciliation` folder. Make sure that the table has at least a column called `place_id` and one called `factgrid_id` that represent the id of the place in the table `gs_places` and in FactGrid respectively. The following cell will load the reconciled places and merge them to the data. If any places remain without a FactGrid id, they will be saved in a new table called `places_without_factgrid.xlsx` in the `reconciliation` folder. Find or create the missing Items in Factgrid and add the information to the `places_reconciled.xlsx` table in the `reconciliation` folder. Afterwards, re-run the workflow. 

In [40]:
# 1. Load the reconciled places
places_reconciled = pd.read_excel("data/reconciliation/places_reconciled.xlsx")[["place_id", "factgrid_id"]]
# 2. Merge them to the table with prepared monasteries
prepared_df = pd.merge(prepared_df, places_reconciled, how="left", on="place_id")
prepared_df = prepared_df.rename(columns={"factgrid_id":"P83"})
prepared_df
# 3. Filter out missing FactGrid Items and store them in a separate table
missing_factgrid_ids = prepared_df[prepared_df['P83'].isna()]
missing_factgrid_ids.to_excel('data/reconciliation/places_without_factGrid.xlsx')
prepared_df = prepared_df.dropna(subset = 'P83')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde,Len,P48,P83
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,51.681667,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778,Q1349608
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,48.563200,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262,Q91532
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,51.295222,NaN,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972,Q90978
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039,Q89258
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n...",NaN,Q1381337
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,49.805300,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317,Q91581
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,52.152778,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611,Q10384
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,53.053089,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455,Q1348304
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,51.666767,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778,Q285683
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,52.916129,NaN,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792,Q32531


## Instance of statement

To state that these items are building complexes, the Item [Q635758](https://database.factgrid.de/wiki/Item:Q635758) (building complex) is connected to all entries using [P2](https://database.factgrid.de/wiki/Property:P2) (instance of)

In [41]:
prepared_df["P2"] = "Q635758"
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,latitude,location_name,id_gsn,status,monastery_name,Lde,Len,P48,P83,P2
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,51.681667,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778,Q1349608,Q635758
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,48.563200,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262,Q91532,Q635758
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,51.295222,NaN,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972,Q90978,Q635758
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,53.863182,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039,Q89258,Q635758
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,NaN,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n...",NaN,Q1381337,Q635758
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,49.805300,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317,Q91581,Q635758
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,52.152778,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611,Q10384,Q635758
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,53.053089,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455,Q1348304,Q635758
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,51.666767,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778,Q285683,Q635758
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,52.916129,NaN,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792,Q32531,Q635758


## Vocabulary Terms

In order to keep a mapping between the monastery database and FactGrid, every item will receive a distinct vocabulary term that is constructed using the `id_monastery_location` from the `gs_monastery_location` table. The FactGrid Property to use is [P1301](https://database.factgrid.de/wiki/Property:P1301) (GS vocabulary term). For the construction, the following pattern is being used: `GSMonasteryLocation<id_monastery_location>`.

In [42]:
prepared_df['P1301'] = prepared_df['id_monastery_location'].apply(lambda x: f'\"GSMonasteryLocation{x}\"')
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,location_name,id_gsn,status,monastery_name,Lde,Len,P48,P83,P2,P1301
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,...,NaN,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778,Q1349608,Q635758,"""GSMonasteryLocation13732"""
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,...,NaN,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262,Q91532,Q635758,"""GSMonasteryLocation8632"""
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,...,NaN,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972,Q90978,Q635758,"""GSMonasteryLocation8408"""
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,...,Neukloster (Kussin),2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039,Q89258,Q635758,"""GSMonasteryLocation1324"""
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,...,Parchow,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n...",NaN,Q1381337,Q635758,"""GSMonasteryLocation1325"""
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,...,Schwarzach am Main,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317,Q91581,Q635758,"""GSMonasteryLocation8299"""
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,...,NaN,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611,Q10384,Q635758,"""GSMonasteryLocation94"""
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,...,NaN,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455,Q1348304,Q635758,"""GSMonasteryLocation13869"""
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,...,NaN,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778,Q285683,Q635758,"""GSMonasteryLocation6137"""
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,...,NaN,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792,Q32531,Q635758,"""GSMonasteryLocation873"""


## Dioceses

By connecting to modern municipalities, it is possible to understand in which territorial structures the (former) building complexes are located today. However, the monastery database also contains information about the historical diocese in which the building complexes were located. This information is stored in the table `gs_places` in the column `diocese_id`. Therefore, the locations where monastery locations are located are assigned to a diocese. In FactGrid, we connect the information about the dioceses directly to the building complexes. A building complex has a property [P1003](https://database.factgrid.de/wiki/Item:Q21662) (Diocese), which connects to a diocese item, for example the Archdiocese of Mainz ([Q153230](https://database.factgrid.de/wiki/Item:Q153230)). The historical affiliation of a location to a diocese is a complex phenomenon. On the one hand, this changed over time, especially in border areas. On the other hand, it is also possible that an area that we understand today as a contiguous location was not a contiguous location around 1500 and only partially belonged to a certain diocese. Therefore, we separate the modern territorial localization (statements about the current location of the address) from the historical localization (statements about the affiliation to a diocese).

In [43]:
# Merge gs_places['diocese_id] to existing table
places_selection = dataframes["gs_places"][["id_places", "diocese_id"]]
diocese_urls_selection = dataframes["gs_id_external_urls_diocese"][dataframes["gs_id_external_urls_diocese"]["url_type_id"]==42][["diocese_id", "url_value"]]
diocese_urls_selection
prepared_df = pd.merge(prepared_df, places_selection, how="left", left_on="place_id", right_on="id_places").drop(columns="id_places")
prepared_df = pd.merge(prepared_df, diocese_urls_selection, how="left", left_on="diocese_id", right_on="diocese_id").drop(columns="diocese_id").rename(columns={"url_value":"P1003"})
prepared_df

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,id_gsn,status,monastery_name,Lde,Len,P48,P83,P2,P1301,P1003
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,...,8502,Online,"Kartause Sionsberg, Noordgouwe, Niederlande","""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778,Q1349608,Q635758,"""GSMonasteryLocation13732""",Q153266
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,...,60372,Online,Franziskanerreformatenkloster Schrobenhausen,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262,Q91532,Q635758,"""GSMonasteryLocation8632""",Q153179
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,...,3790,Online,Zisterzienserinnenkloster Roßleben,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972,Q90978,Q635758,"""GSMonasteryLocation8408""",Q153222
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,...,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039,Q89258,Q635758,"""GSMonasteryLocation1324""",Q153242
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,...,2055,Online,"Benediktinerinnen-, Zisterzienserinnenkloster ...","""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n...",NaN,Q1381337,Q635758,"""GSMonasteryLocation1325""",Q153242
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,...,60271,Online,Benediktinerabtei Münsterschwarzach,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317,Q91581,Q635758,"""GSMonasteryLocation8299""",Q153247
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,...,93,Online,"Benediktinerabtei St. Michael, Hildesheim","""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611,Q10384,Q635758,"""GSMonasteryLocation94""",Q153224
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,...,8609,Online,"Augustinerchorherrenstift Tirns, Niederlande","""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455,Q1348304,Q635758,"""GSMonasteryLocation13869""",Q153266
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,...,3768,Online,Dominikanerinnenkloster Wiederstedt,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778,Q285683,Q635758,"""GSMonasteryLocation6137""",Q153222
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,...,814,Online,"Kollegiatstift St. Andreas, Verden","""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792,Q32531,Q635758,"""GSMonasteryLocation873""",Q153245


## Sources / References

Every Statement in FactGrid should be supported by a Source/Reference. To achieve this, a source column `S471` is added after each relevant property to link to the Monastery Database Entries using the Property [P471](https://database.factgrid.de/wiki/Property:P471).

In [44]:
final_table = prepared_df.copy()
for colname in ["P48", "P83", "P1003"]:
    final_table.insert(final_table.columns.get_loc(colname)+1, "S471", final_table["gsn_id"].apply(lambda x:f'\"{x}\"'), allow_duplicates=True)
final_table["P131"] = "Q153178"
final_table

,id_monastery_location,place_id,gsn_id,location_begin_tpq,location_begin_taq,location_begin_note,location_end_tpq,location_end_taq,location_end_note,longitude,...,Len,P48,S471,P83,S471,P2,P1301,P1003,S471,P131
0,13732,46483081,8502,1436,NaN,vor 1436,1574,NaN,NaN,3.945278,...,"""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778,"""8502""",Q1349608,"""8502""",Q635758,"""GSMonasteryLocation13732""",Q153266,"""8502""",Q153178
1,8632,10682,60372,1644,NaN,NaN,1802,NaN,NaN,11.262000,...,"""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262,"""60372""",Q91532,"""60372""",Q635758,"""GSMonasteryLocation8632""",Q153179,"""60372""",Q153178
2,8408,18263,3790,1100,1140.0,zwischen 1100 und 1140,1553,NaN,letzte Erwähnung 1553,11.427972,...,"""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972,"""3790""",Q90978,"""3790""",Q635758,"""GSMonasteryLocation8408""",Q153222,"""3790""",Q153178
3,1324,13832,2055,1219,NaN,NaN,1555,NaN,NaN,11.685714,...,"""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039,"""2055""",Q89258,"""2055""",Q635758,"""GSMonasteryLocation1324""",Q153242,"""2055""",Q153178
4,1325,41955,2055,1205,1210.0,um 1210 (?),1219,NaN,NaN,NaN,...,"""Building complex Benedictine and Cistercian n...",NaN,"""2055""",Q1381337,"""2055""",Q635758,"""GSMonasteryLocation1325""",Q153242,"""2055""",Q153178
5,8299,10757,60271,762,782.0,vor 783,1803,NaN,NaN,10.231700,...,"""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317,"""60271""",Q91581,"""60271""",Q635758,"""GSMonasteryLocation8299""",Q153247,"""60271""",Q153178
6,94,5035,93,1005,1022.0,ca. 1010/1022,1803,NaN,NaN,9.943611,...,"""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611,"""93""",Q10384,"""93""",Q635758,"""GSMonasteryLocation94""",Q153224,"""93""",Q153178
7,13869,46483177,8609,1406,NaN,NaN,1572,1580.0,1572/1580,5.636605,...,"""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455,"""8609""",Q1348304,"""8609""",Q635758,"""GSMonasteryLocation13869""",Q153266,"""8609""",Q153178
8,6137,17237,3768,1256,1260.0,kurz nach 1256,1525,1550.0,zwischen 1525 und 1550,11.529778,...,"""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778,"""3768""",Q285683,"""3768""",Q635758,"""GSMonasteryLocation6137""",Q153222,"""3768""",Q153178
9,873,12105,814,1220,NaN,NaN,1649,NaN,NaN,9.228792,...,"""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792,"""814""",Q32531,"""814""",Q635758,"""GSMonasteryLocation873""",Q153245,"""814""",Q153178


## Finalizing

To finalize, the table is cleaned up and transformed into a variety of formats. Most importantly, you will find the V1-statements to create the new building complex items under `data/results/building_complexes/import_building_complexes.tsv`

In [45]:
from helper_functions import df_to_qs_v1
final_table = final_table.drop(columns=["id_monastery_location", "place_id", "gsn_id", "location_begin_tpq", "location_begin_taq", "location_begin_note", "location_end_tpq", "location_end_taq", "location_end_note", "longitude", "latitude", "location_name", "id_gsn", "status", "monastery_name"])
final_table.insert(0, "qid", np.nan)
final_table.to_excel("data/results/building_complexes/import_building_complexes.xlsx", index=False)
final_table.to_csv("data/results/building_complexes/import_building_complexes.csv", index=False, doublequote=False, quoting=csv.QUOTE_NONE, escapechar="§") #hack to save in Quickstatements-applicable format
with open("data/results/building_complexes/import_building_complexes.csv", "r") as file:
    s = file.read()
with open("data/results/building_complexes/import_building_complexes.csv", "w") as file:
    file.write(s.replace("§", ""))
with open("data/results/building_complexes/import_building_complexes.tsv", "w") as file:
    file.write(df_to_qs_v1(final_table))
final_table

,qid,Lde,Len,P48,S471,P83,S471.1,P2,P1301,P1003,S471.2,P131
0,NaN,"""Gebäudekomplex Kartause Sionsberg, Noordgouwe...","""Building complex Carthusian monastery of Sion...",@51.68166667/3.945277778,"""8502""",Q1349608,"""8502""",Q635758,"""GSMonasteryLocation13732""",Q153266,"""8502""",Q153178
1,NaN,"""Gebäudekomplex Franziskanerreformatenkloster ...","""Building complex Franciscans of Schrobenhausen""",@48.5632/11.262,"""60372""",Q91532,"""60372""",Q635758,"""GSMonasteryLocation8632""",Q153179,"""60372""",Q153178
2,NaN,"""Gebäudekomplex Zisterzienserinnenkloster Roßl...","""Building complex of Cistercian nunnery Roßleben""",@51.295222/11.427972,"""3790""",Q90978,"""3790""",Q635758,"""GSMonasteryLocation8408""",Q153222,"""3790""",Q153178
3,NaN,"""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine nuns, Cistercian...",@53.863182068216/11.685713838039,"""2055""",Q89258,"""2055""",Q635758,"""GSMonasteryLocation1324""",Q153242,"""2055""",Q153178
4,NaN,"""Gebäudekomplex Benediktinerinnen-, Zisterzien...","""Building complex Benedictine and Cistercian n...",NaN,"""2055""",Q1381337,"""2055""",Q635758,"""GSMonasteryLocation1325""",Q153242,"""2055""",Q153178
5,NaN,"""Gebäudekomplex Benediktinerabtei Münsterschwa...","""Building complex Benedictine abbey Münstersch...",@49.8053/10.2317,"""60271""",Q91581,"""60271""",Q635758,"""GSMonasteryLocation8299""",Q153247,"""60271""",Q153178
6,NaN,"""Gebäudekomplex Benediktinerabtei St. Michael,...","""Building complex Benedictine abbey of St. Mic...",@52.152778/9.943611,"""93""",Q10384,"""93""",Q635758,"""GSMonasteryLocation94""",Q153224,"""93""",Q153178
7,NaN,"""Gebäudekomplex Augustinerchorherrenstift Tirn...","""Building complex Canons Regular of St Augusti...",@53.053089427208/5.63660450368455,"""8609""",Q1348304,"""8609""",Q635758,"""GSMonasteryLocation13869""",Q153266,"""8609""",Q153178
8,NaN,"""Gebäudekomplex Dominikanerinnenkloster Wieder...","""Building complex Dominican Nuns' monastery of...",@51.666767/11.529778,"""3768""",Q285683,"""3768""",Q635758,"""GSMonasteryLocation6137""",Q153222,"""3768""",Q153178
9,NaN,"""Gebäudekomplex Kollegiatstift St. Andreas, Ve...","""Building complex Collegiate Church St. Andrea...",@52.916129/9.228792,"""814""",Q32531,"""814""",Q635758,"""GSMonasteryLocation873""",Q153245,"""814""",Q153178


## Next steps
As a next step, you should run notebook 2 - Monasteries to create the religious community items that go together with the building complexes. Afterwards you can copy the V1 statements from both, `data/results/building_complexes/import_building_complexes.csv` and `data/results/monasteries/import_monasteries.csv` to Quickstatements and upload.